In [4]:
import findspark
findspark.init()

In [7]:
from pyspark import SparkConf, SparkContext

In [8]:
conf = SparkConf().setMaster("local[4]").setAppName("Bro-Var")
sc = SparkContext(conf = conf).getOrCreate()

# Read Products and Return product_id,product_name

In [9]:
def read_products():
    products_text_wrapper = open("C:\\Users\\Muhammed\\Desktop\\Spark\\6\\products.txt", "r", encoding="utf-8")
    
    #read line
    products = products_text_wrapper.readlines()
    
    product_id_name ={}
    
    for line in products:
        #delete header
        if "productCategoryId" not in line:
            product_id = int(line.split(",")[0])
            product_name = line.split(",")[2]
            
            product_id_name.update({product_id:product_name})
    return product_id_name        

In [10]:
products = read_products()

In [11]:
broadcast_products = sc.broadcast(products)

In [12]:
broadcast_products.value.get(10)

"Under Armour Men's Highlight MC Football Clea"

# Read order_item and create RDD

In [14]:
order_items_rdd = sc.textFile("C:\\Users\\Muhammed\\Desktop\\Spark\\6\\order_items.txt") \
.filter(lambda x: "orderItemOrderId" not in x)

In [15]:
order_items_rdd.take(5)

['1,1,957,1,299.98,299.98',
 '2,2,1073,1,199.99,199.99',
 '3,2,502,5,250.0,50.0',
 '4,2,403,1,129.99,129.99',
 '5,4,897,2,49.98,24.99']

# make order_items  pairRDD

In [16]:
def make_order_items_pairRDD(line):
    order_item_product_id = int(line.split(",")[2])
    order_item_sub_total = float(line.split(",")[4])
    
    return(order_item_product_id, order_item_sub_total)

In [17]:
order_items_pairRDD = order_items_rdd.map(make_order_items_pairRDD)

In [18]:
order_items_pairRDD.take(5)

[(957, 299.98), (1073, 199.99), (502, 250.0), (403, 129.99), (897, 49.98)]

In [26]:
# sort by sub total
sorted_orders = order_items_pairRDD.reduceByKey(lambda x,y: x+y) \
.map(lambda x: (x[1], x[0])) \
.sortByKey(False) \
.map(lambda x: (x[1], x[0]))

In [27]:
sorted_orders.take(5)

[(1004, 6929653.499999708),
 (365, 4421143.019999639),
 (957, 4118425.419999785),
 (191, 3667633.1999997487),
 (502, 3147800.0)]

# Combine order_items(pairRDD) and products(broadcast variable)

In [28]:
sorted_orders_with_product_name = sorted_orders.map(lambda x: (broadcast_products.value.get(x[0]),x[1]))

In [30]:
sorted_orders_with_product_name.take(20)

[('Field & Stream Sportsman 16 Gun Fire Safe', 6929653.499999708),
 ('Perfect Fitness Perfect Rip Deck', 4421143.019999639),
 ("Diamondback Women's Serene Classic Comfort Bi", 4118425.419999785),
 ("Nike Men's Free 5.0+ Running Shoe", 3667633.1999997487),
 ("Nike Men's Dri-FIT Victory Golf Polo", 3147800.0),
 ('Pelican Sunstream 100 Kayak', 3099844.999999871),
 ("Nike Men's CJ Elite 2 TD Football Cleat", 2891757.5399998166),
 ("O'Brien Men's Neoprene Life Vest", 2888993.9399996493),
 ("Under Armour Girls' Toddler Spine Surge Runni", 1269082.649999932),
 ('adidas Youth Germany Black/Red Away Match Soc', 67830.0),
 ("LIJA Women's Eyelet Sleeveless Golf Polo", 61490.0),
 ("adidas Men's F10 Messi TRX FG Soccer Cleat", 56330.61000000003),
 ('Titleist Pro V1x High Numbers Personalized Go', 47206.920000000006),
 ("Under Armour Women's Micro G Skulpt Running S", 46559.58999999997),
 ("Nike Men's Comfort 2 Slide", 44585.09000000002),
 ('Titleist Pro V1 High Numbers Personalized Gol', 44243.4900